# Script to benchmark WKT vs WKT vs PointXY


fetch timedelta tgeompoints, 

# WKB

results Desktop:

1. MMSI list fetched in 0.03388237953186035 seconds
2. min/max ts fetched in 0.5566732883453369 seconds
3. Time delta Tgeompoints fetched in 0.5069243907928467 seconds
4. Vector Layer created in 0.00994873046875 seconds
5. Features created in 0.043292999267578125 seconds
6. Average FPS : 15.061229102429822
7. On_new_frame average time : 0.06774496078491211
8. Value_at_timestamp average time : 0.06359884262084961
9. Updating geometries average time : 0.0041461181640625

In [ ]:
from enum import Enum
from datetime import timedelta
from pymeos.db.psycopg import MobilityDB
from pymeos import *
from shapely.geometry import Point
import time


class Time_granularity(Enum):
    # MILLISECOND = {"timedelta" : timedelta(milliseconds=1), "qgs_unit" : QgsUnitTypes.TemporalUnit.Milliseconds, "name" : "MILLISECOND"}
    SECOND = {"timedelta" : timedelta(seconds=1), "name" : "SECOND", "steps" : 1}
    MINUTE = {"timedelta" : timedelta(minutes=1), "name" : "MINUTE", "steps" : 1}
    # HOUR = {"timedelta" : timedelta(hours=1), "qgs_unit" : QgsUnitTypes.TemporalUnit.Hours, "name" : "HOUR"}

    @classmethod
    def set_time_step(cls, steps):
        Time_granularity.SECOND.value["timedelta"] = timedelta(seconds=steps)
        Time_granularity.SECOND.value["steps"] = steps
        Time_granularity.MINUTE.value["timedelta"] = timedelta(minutes=steps)
        Time_granularity.MINUTE.value["steps"] = steps
        return cls
    
SRID = 4326
DATABASE_NAME = "mobilitydb"
TPOINT_TABLE_NAME = "PyMEOS_demo"
TPOINT_ID_COLUMN_NAME = "MMSI"
TPOINT_COLUMN_NAME = "trajectory"
GRANULARITY = Time_granularity.set_time_step(1).MINUTE

connection_parameters = {
                'host': "localhost",
                'port': 5432,
                'dbname': DATABASE_NAME,
                'user': "postgres",
                'password': "postgres",
                'table_name': TPOINT_TABLE_NAME,
                'id_column_name': TPOINT_ID_COLUMN_NAME,
                'tpoint_column_name': TPOINT_COLUMN_NAME,
            }


######  MobilityDB connection ######
now = time.time()
pymeos_initialize()

connection_params = {
            "host": connection_parameters["host"],
            "port": connection_parameters["port"],
            "dbname": connection_parameters["dbname"],
            "user": connection_parameters["user"],
            "password": connection_parameters["password"],
            }


table_name = connection_parameters["table_name"]
id_column_name = connection_parameters["id_column_name"]
tpoint_column_name = connection_parameters["tpoint_column_name"]     
connection = MobilityDB.connect(**connection_params)

######  Fetching MMSI list  ######    
cursor = connection.cursor()
query = f"""
        SELECT {id_column_name} FROM public.{table_name} ;
        """

cursor.execute(query)
                
ids_list = cursor.fetchall()
now_2 = time.time()
print(f"1. MMSI list fetched in {now_2 - now} seconds")

objects_count = len(ids_list)
x_min,y_min, x_max, y_max = [-180, -90, 180, 90]
ids_list = [ f"'{id[0]}'"  for id in ids_list]
objects_id_str = ', '.join(map(str, ids_list))

######  Fetching min/max timestamp of dataset ######

cursor.execute(f"SELECT MIN(startTimestamp({tpoint_column_name})) AS earliest_timestamp FROM public.{table_name};")
min_ts =cursor.fetchone()[0]

cursor.execute(f"SELECT MAX(endTimestamp({tpoint_column_name})) AS latest_timestamp FROM public.{table_name};")
max_ts  = cursor.fetchone()[0]


end_ts = min_ts + (GRANULARITY.value["timedelta"] * 25)

now_3 = time.time()
print(f"2. min/max ts fetched in {now_3 - now_2} seconds")

######  Fetching Tgeompoints for subset 0 - 25 ######
query = f"""WITH trajectories as (
        SELECT 
            atStbox(
                a.{tpoint_column_name}::tgeompoint,
                stbox(
                    ST_MakeEnvelope(
                        {x_min}, {y_min}, -- xmin, ymin
                        {x_max}, {y_max}, -- xmax, ymax
                        {SRID} -- SRID
                    ),
                    tstzspan('[{min_ts}, {end_ts}]')
                )
            ) as trajectory
        FROM public.{table_name} as a 
        WHERE a.{id_column_name} in ({objects_id_str}))
    
        SELECT
                rs.trajectory
        FROM trajectories as rs ;"""

cursor.execute(query)
# print(f"query : {query}\n")
rows = cursor.fetchall()
now_4 = time.time()
print(f"3. Time delta Tgeompoints fetched in {now_4 - now_3} seconds")

######  Creating Vector Layer ######

vlayer = QgsVectorLayer("Point", "Benchmarking", "memory")
pr = vlayer.dataProvider()
pr.addAttributes([ QgsField("id", QVariant.String) ,QgsField("start_time", QVariant.DateTime), QgsField("end_time", QVariant.DateTime)])
vlayer.updateFields()
tp = vlayer.temporalProperties()
tp.setIsActive(True)
tp.setMode(qgis.core.QgsVectorLayerTemporalProperties.ModeFeatureDateTimeStartAndEndFromFields)
tp.setStartField("start_time")
tp.setEndField("end_time")
QgsProject.instance().addMapLayer(vlayer)

iface.mapCanvas().setDestinationCrs(QgsCoordinateReferenceSystem(f"EPSG:{SRID}"))
now_5 = time.time() 
print(f"4. Vector Layer created in {now_5 - now_4} seconds")

######  Creating QgisFeatures  ######    


features_list =[]
vfields = vlayer.fields()
start_datetime_obj = QDateTime(min_ts)
end_datetime_obj = QDateTime(max_ts)


geometries = {}
for i in range(1, objects_count+1):
    feat = QgsFeature(vfields)
    feat.setAttributes([ids_list[i-1],start_datetime_obj, end_datetime_obj])
    geom = QgsGeometry()
    geometries[i] = geom
    feat.setGeometry(geom)
    features_list.append(feat)

pr.addFeatures(features_list)
now_6 = time.time()
print(f"5. Features created in {now_6 - now_5} seconds")

######  On_new_frame  performance ######

on_new_frames_records = []
value_at_timestamp_records = []
updating_geometries_records = []

empty_point_wkb = Point().wkb

for frame in range(25):
    now_7 = time.time()
    timestamp = min_ts + GRANULARITY.value["timedelta"]  * frame
    
    for i in range(1, objects_count+1):
        try:
            position = rows[i-1][0].value_at_timestamp(timestamp)
            geometries[i].fromWkb(position.wkb) 
        except:
            geometries[i].fromWkb(empty_point_wkb)

    now_8 = time.time()
    value_at_timestamp_records.append(now_8 - now_7)

    vlayer.startEditing()
    pr.changeGeometryValues(geometries) # Updating geometries for all features
    vlayer.commitChanges()
    iface.vectorLayerTools().stopEditing(vlayer)
    now_9 = time.time()
    updating_geometries_records.append(now_9 - now_8)
    on_new_frames_records.append(now_9 - now_7)


fps_record = [1/frame_time for frame_time in on_new_frames_records]

average_fps = sum(fps_record)/len(fps_record)

print(f"6. Average FPS : {average_fps}")
print(f"7. On_new_frame average time : {sum(on_new_frames_records) / len(on_new_frames_records)}")
print(f"8. Value_at_timestamp average time : {sum(value_at_timestamp_records) / len(value_at_timestamp_records)}")
print(f"9. Updating geometries average time : {sum(updating_geometries_records) / len(updating_geometries_records)}")



# WKT


results Desktop:
1. MMSI list fetched in 0.03459334373474121 seconds
2. min/max ts fetched in 0.5492024421691895 seconds
3. Time delta Tgeompoints fetched in 0.49392223358154297 seconds
4. Vector Layer created in 0.01004338264465332 seconds
5. Features created in 0.04014229774475098 seconds
6. Average FPS : 12.501380429277749
7. On_new_frame average time : 0.08231148719787598
8. Value_at_timestamp average time : 0.07838180541992187
9. Updating geometries average time : 0.003929681777954101

In [ ]:
from enum import Enum
from datetime import timedelta
from pymeos.db.psycopg import MobilityDB
from pymeos import *
from shapely.geometry import Point
import time


class Time_granularity(Enum):
    # MILLISECOND = {"timedelta" : timedelta(milliseconds=1), "qgs_unit" : QgsUnitTypes.TemporalUnit.Milliseconds, "name" : "MILLISECOND"}
    SECOND = {"timedelta" : timedelta(seconds=1), "name" : "SECOND", "steps" : 1}
    MINUTE = {"timedelta" : timedelta(minutes=1), "name" : "MINUTE", "steps" : 1}
    # HOUR = {"timedelta" : timedelta(hours=1), "qgs_unit" : QgsUnitTypes.TemporalUnit.Hours, "name" : "HOUR"}

    @classmethod
    def set_time_step(cls, steps):
        Time_granularity.SECOND.value["timedelta"] = timedelta(seconds=steps)
        Time_granularity.SECOND.value["steps"] = steps
        Time_granularity.MINUTE.value["timedelta"] = timedelta(minutes=steps)
        Time_granularity.MINUTE.value["steps"] = steps
        return cls
    
SRID = 4326
DATABASE_NAME = "mobilitydb"
TPOINT_TABLE_NAME = "PyMEOS_demo"
TPOINT_ID_COLUMN_NAME = "MMSI"
TPOINT_COLUMN_NAME = "trajectory"
GRANULARITY = Time_granularity.set_time_step(1).MINUTE

connection_parameters = {
                'host': "localhost",
                'port': 5432,
                'dbname': DATABASE_NAME,
                'user': "postgres",
                'password': "postgres",
                'table_name': TPOINT_TABLE_NAME,
                'id_column_name': TPOINT_ID_COLUMN_NAME,
                'tpoint_column_name': TPOINT_COLUMN_NAME,
            }



######  MobilityDB connection ######
now = time.time()
pymeos_initialize()

connection_params = {
            "host": connection_parameters["host"],
            "port": connection_parameters["port"],
            "dbname": connection_parameters["dbname"],
            "user": connection_parameters["user"],
            "password": connection_parameters["password"],
            }


table_name = connection_parameters["table_name"]
id_column_name = connection_parameters["id_column_name"]
tpoint_column_name = connection_parameters["tpoint_column_name"]     
connection = MobilityDB.connect(**connection_params)

######  Fetching MMSI list  ######    
cursor = connection.cursor()
query = f"""
        SELECT {id_column_name} FROM public.{table_name} ;
        """

cursor.execute(query)
                
ids_list = cursor.fetchall()
now_2 = time.time()
print(f"1. MMSI list fetched in {now_2 - now} seconds")

objects_count = len(ids_list)
x_min,y_min, x_max, y_max = [-180, -90, 180, 90]
ids_list = [ f"'{id[0]}'"  for id in ids_list]
objects_id_str = ', '.join(map(str, ids_list))

######  Fetching min/max timestamp of dataset ######

cursor.execute(f"SELECT MIN(startTimestamp({tpoint_column_name})) AS earliest_timestamp FROM public.{table_name};")
min_ts =cursor.fetchone()[0]

cursor.execute(f"SELECT MAX(endTimestamp({tpoint_column_name})) AS latest_timestamp FROM public.{table_name};")
max_ts  = cursor.fetchone()[0]


end_ts = min_ts + (GRANULARITY.value["timedelta"] * 25)

now_3 = time.time()
print(f"2. min/max ts fetched in {now_3 - now_2} seconds")

######  Fetching Tgeompoints for subset 0 - 25 ######
query = f"""WITH trajectories as (
        SELECT 
            atStbox(
                a.{tpoint_column_name}::tgeompoint,
                stbox(
                    ST_MakeEnvelope(
                        {x_min}, {y_min}, -- xmin, ymin
                        {x_max}, {y_max}, -- xmax, ymax
                        {SRID} -- SRID
                    ),
                    tstzspan('[{min_ts}, {end_ts}]')
                )
            ) as trajectory
        FROM public.{table_name} as a 
        WHERE a.{id_column_name} in ({objects_id_str}))
    
        SELECT
                rs.trajectory
        FROM trajectories as rs ;"""

cursor.execute(query)
# print(f"query : {query}\n")
rows = cursor.fetchall()
now_4 = time.time()
print(f"3. Time delta Tgeompoints fetched in {now_4 - now_3} seconds")

######  Creating Vector Layer ######

vlayer = QgsVectorLayer("Point", "Benchmarking", "memory")
pr = vlayer.dataProvider()
pr.addAttributes([ QgsField("id", QVariant.String) ,QgsField("start_time", QVariant.DateTime), QgsField("end_time", QVariant.DateTime)])
vlayer.updateFields()
tp = vlayer.temporalProperties()
tp.setIsActive(True)
tp.setMode(qgis.core.QgsVectorLayerTemporalProperties.ModeFeatureDateTimeStartAndEndFromFields)
tp.setStartField("start_time")
tp.setEndField("end_time")
QgsProject.instance().addMapLayer(vlayer)

iface.mapCanvas().setDestinationCrs(QgsCoordinateReferenceSystem(f"EPSG:{SRID}"))
now_5 = time.time() 
print(f"4. Vector Layer created in {now_5 - now_4} seconds")

######  Creating QgisFeatures  ######    


features_list =[]
vfields = vlayer.fields()
start_datetime_obj = QDateTime(min_ts)
end_datetime_obj = QDateTime(max_ts)


geometries = {}
for i in range(1, objects_count+1):
    feat = QgsFeature(vfields)
    feat.setAttributes([ids_list[i-1],start_datetime_obj, end_datetime_obj])
    geom = QgsGeometry()
    geometries[i] = geom
    feat.setGeometry(geom)
    features_list.append(feat)

pr.addFeatures(features_list)
now_6 = time.time()
print(f"5. Features created in {now_6 - now_5} seconds")

######  On_new_frame  performance ######

on_new_frames_records = []
value_at_timestamp_records = []
updating_geometries_records = []

empty_point_wkt = Point().wkt

for frame in range(25):
    now_7 = time.time()
    timestamp = min_ts + GRANULARITY.value["timedelta"]  * frame
    
    for i in range(1, objects_count+1):
        try:
            position = rows[i-1][0].value_at_timestamp(timestamp)
            geometries[i] = QgsGeometry().fromWkt(position.wkt) 
        except:
            geometries[i]= QgsGeometry().fromWkt(empty_point_wkt)

    now_8 = time.time()
    value_at_timestamp_records.append(now_8 - now_7)

    vlayer.startEditing()
    pr.changeGeometryValues(geometries) # Updating geometries for all features
    vlayer.commitChanges()
    iface.vectorLayerTools().stopEditing(vlayer)
    now_9 = time.time()
    updating_geometries_records.append(now_9 - now_8)
    on_new_frames_records.append(now_9 - now_7)


fps_record = [1/frame_time for frame_time in on_new_frames_records]

average_fps = sum(fps_record)/len(fps_record)

print(f"6. Average FPS : {average_fps}")
print(f"7. On_new_frame average time : {sum(on_new_frames_records) / len(on_new_frames_records)}")
print(f"8. Value_at_timestamp average time : {sum(value_at_timestamp_records) / len(value_at_timestamp_records)}")
print(f"9. Updating geometries average time : {sum(updating_geometries_records) / len(updating_geometries_records)}")



# Point XY

results Desktop:

1. MMSI list fetched in 0.033403635025024414 seconds
2. min/max ts fetched in 0.5576505661010742 seconds
3. Time delta Tgeompoints fetched in 0.5065641403198242 seconds
4. Vector Layer created in 0.012407302856445312 seconds
5. Features created in 0.043254852294921875 seconds
6. Average FPS : 14.252336723148323
7. On_new_frame average time : 0.07167965888977051
8. Value_at_timestamp average time : 0.06773480415344238
9. Updating geometries average time : 0.003944854736328125


In [ ]:
from enum import Enum
from datetime import timedelta
from pymeos.db.psycopg import MobilityDB
from pymeos import *
from shapely.geometry import Point
import time


class Time_granularity(Enum):
    # MILLISECOND = {"timedelta" : timedelta(milliseconds=1), "qgs_unit" : QgsUnitTypes.TemporalUnit.Milliseconds, "name" : "MILLISECOND"}
    SECOND = {"timedelta" : timedelta(seconds=1), "name" : "SECOND", "steps" : 1}
    MINUTE = {"timedelta" : timedelta(minutes=1), "name" : "MINUTE", "steps" : 1}
    # HOUR = {"timedelta" : timedelta(hours=1), "qgs_unit" : QgsUnitTypes.TemporalUnit.Hours, "name" : "HOUR"}

    @classmethod
    def set_time_step(cls, steps):
        Time_granularity.SECOND.value["timedelta"] = timedelta(seconds=steps)
        Time_granularity.SECOND.value["steps"] = steps
        Time_granularity.MINUTE.value["timedelta"] = timedelta(minutes=steps)
        Time_granularity.MINUTE.value["steps"] = steps
        return cls
    
SRID = 4326
DATABASE_NAME = "mobilitydb"
TPOINT_TABLE_NAME = "PyMEOS_demo"
TPOINT_ID_COLUMN_NAME = "MMSI"
TPOINT_COLUMN_NAME = "trajectory"
GRANULARITY = Time_granularity.set_time_step(1).MINUTE

connection_parameters = {
                'host': "localhost",
                'port': 5432,
                'dbname': DATABASE_NAME,
                'user': "postgres",
                'password': "postgres",
                'table_name': TPOINT_TABLE_NAME,
                'id_column_name': TPOINT_ID_COLUMN_NAME,
                'tpoint_column_name': TPOINT_COLUMN_NAME,
            }


######  MobilityDB connection ######
now = time.time()
pymeos_initialize()

connection_params = {
            "host": connection_parameters["host"],
            "port": connection_parameters["port"],
            "dbname": connection_parameters["dbname"],
            "user": connection_parameters["user"],
            "password": connection_parameters["password"],
            }


table_name = connection_parameters["table_name"]
id_column_name = connection_parameters["id_column_name"]
tpoint_column_name = connection_parameters["tpoint_column_name"]     
connection = MobilityDB.connect(**connection_params)

######  Fetching MMSI list  ######    
cursor = connection.cursor()
query = f"""
        SELECT {id_column_name} FROM public.{table_name} ;
        """

cursor.execute(query)
                
ids_list = cursor.fetchall()
now_2 = time.time()
print(f"1. MMSI list fetched in {now_2 - now} seconds")

objects_count = len(ids_list)
x_min,y_min, x_max, y_max = [-180, -90, 180, 90]
ids_list = [ f"'{id[0]}'"  for id in ids_list]
objects_id_str = ', '.join(map(str, ids_list))

######  Fetching min/max timestamp of dataset ######

cursor.execute(f"SELECT MIN(startTimestamp({tpoint_column_name})) AS earliest_timestamp FROM public.{table_name};")
min_ts =cursor.fetchone()[0]

cursor.execute(f"SELECT MAX(endTimestamp({tpoint_column_name})) AS latest_timestamp FROM public.{table_name};")
max_ts  = cursor.fetchone()[0]


end_ts = min_ts + (GRANULARITY.value["timedelta"] * 25)

now_3 = time.time()
print(f"2. min/max ts fetched in {now_3 - now_2} seconds")

######  Fetching Tgeompoints for subset 0 - 25 ######
query = f"""WITH trajectories as (
        SELECT 
            atStbox(
                a.{tpoint_column_name}::tgeompoint,
                stbox(
                    ST_MakeEnvelope(
                        {x_min}, {y_min}, -- xmin, ymin
                        {x_max}, {y_max}, -- xmax, ymax
                        {SRID} -- SRID
                    ),
                    tstzspan('[{min_ts}, {end_ts}]')
                )
            ) as trajectory
        FROM public.{table_name} as a 
        WHERE a.{id_column_name} in ({objects_id_str}))
    
        SELECT
                rs.trajectory
        FROM trajectories as rs ;"""

cursor.execute(query)
# print(f"query : {query}\n")
rows = cursor.fetchall()
now_4 = time.time()
print(f"3. Time delta Tgeompoints fetched in {now_4 - now_3} seconds")

######  Creating Vector Layer ######

vlayer = QgsVectorLayer("Point", "Benchmarking", "memory")
pr = vlayer.dataProvider()
pr.addAttributes([ QgsField("id", QVariant.String) ,QgsField("start_time", QVariant.DateTime), QgsField("end_time", QVariant.DateTime)])
vlayer.updateFields()
tp = vlayer.temporalProperties()
tp.setIsActive(True)
tp.setMode(qgis.core.QgsVectorLayerTemporalProperties.ModeFeatureDateTimeStartAndEndFromFields)
tp.setStartField("start_time")
tp.setEndField("end_time")
QgsProject.instance().addMapLayer(vlayer)

iface.mapCanvas().setDestinationCrs(QgsCoordinateReferenceSystem(f"EPSG:{SRID}"))
now_5 = time.time() 
print(f"4. Vector Layer created in {now_5 - now_4} seconds")

######  Creating QgisFeatures  ######    


features_list =[]
vfields = vlayer.fields()
start_datetime_obj = QDateTime(min_ts)
end_datetime_obj = QDateTime(max_ts)


geometries = {}
for i in range(1, objects_count+1):
    feat = QgsFeature(vfields)
    feat.setAttributes([ids_list[i-1],start_datetime_obj, end_datetime_obj])
    geom = QgsGeometry()
    geometries[i] = geom
    feat.setGeometry(geom)
    features_list.append(feat)

pr.addFeatures(features_list)
now_6 = time.time()
print(f"5. Features created in {now_6 - now_5} seconds")

######  On_new_frame  performance ######

on_new_frames_records = []
value_at_timestamp_records = []
updating_geometries_records = []

# empty_point_wkt = Point().wkt

for frame in range(25):
    now_7 = time.time()
    timestamp = min_ts + GRANULARITY.value["timedelta"]  * frame
    
    for i in range(1, objects_count+1):
        try:
            position = rows[i-1][0].value_at_timestamp(timestamp)
            x, y = position.x, position.y
            geometries[i] = QgsGeometry.fromPointXY(QgsPointXY(x,y))
        except:
            geometries[i] = QgsGeometry()

    now_8 = time.time()
    value_at_timestamp_records.append(now_8 - now_7)

    vlayer.startEditing()
    pr.changeGeometryValues(geometries) # Updating geometries for all features
    vlayer.commitChanges()
    iface.vectorLayerTools().stopEditing(vlayer)
    now_9 = time.time()
    updating_geometries_records.append(now_9 - now_8)
    on_new_frames_records.append(now_9 - now_7)


fps_record = [1/frame_time for frame_time in on_new_frames_records]

average_fps = sum(fps_record)/len(fps_record)

print(f"6. Average FPS : {average_fps}")
print(f"7. On_new_frame average time : {sum(on_new_frames_records) / len(on_new_frames_records)}")
print(f"8. Value_at_timestamp average time : {sum(value_at_timestamp_records) / len(value_at_timestamp_records)}")
print(f"9. Updating geometries average time : {sum(updating_geometries_records) / len(updating_geometries_records)}")

